In [1]:
import pandas as pd

In [2]:
df_table_all = pd.read_csv('Data/original_data_files/final_abundance_table_v46_rarefied.csv', index_col=0)

In [3]:
df_table_all

,GlobalSpecies0001,GlobalSpecies0002,GlobalSpecies0003,GlobalSpecies0004,GlobalSpecies0005,GlobalSpecies0006,GlobalSpecies0007,GlobalSpecies0008,GlobalSpecies0009,GlobalSpecies0010,...,GlobalSpecies0991,GlobalSpecies0992,GlobalSpecies0993,GlobalSpecies0994,GlobalSpecies0995,GlobalSpecies0996,GlobalSpecies0997,GlobalSpecies0998,GlobalSpecies0999,GlobalSpecies1000
Sample1,11,0,0,0,0,0,0,0,0,0,...,31,0,0,0,0,0,0,0,9,0
Sample2,12,0,0,0,0,0,0,11,0,0,...,0,0,0,0,0,2,2,0,0,0
Sample3,0,0,0,0,0,0,0,0,0,0,...,0,14,0,0,0,29,0,0,0,0
Sample4,0,0,0,0,0,0,0,0,0,0,...,34,0,0,0,5,0,0,0,0,0
Sample5,0,0,0,0,0,0,0,2,0,0,...,0,0,22,0,1,5,0,5,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sample999996,0,0,0,0,0,0,0,11,0,0,...,0,0,0,0,0,0,0,0,0,0
Sample999997,0,2,0,0,0,0,0,26,0,0,...,0,0,0,0,0,4,0,0,0,0
Sample999998,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sample999999,0,0,0,0,0,0,0,3,0,12,...,17,0,0,0,0,0,0,7,1,0


In [5]:
import pandas as pd
import numpy as np
import biom
import os
import h5py # BIOM HDF5 格式通常需要这个
from sklearn.utils import shuffle as sk_shuffle # 用于打乱索引

def create_and_save_biom_subsets(
    df_abundance: pd.DataFrame,
    subset_size: int,
    num_subsets: int = 5,
    output_folder: str = "biom_subsets",
    random_seed: int = 42
) -> list:
    """
    对输入的丰度表DataFrame进行预处理，随机抽取指定数量和大小的子集，
    并将每个子集以BIOM格式保存到指定文件夹。

    参数:
    df_abundance (pd.DataFrame): 输入的丰度表，行为样本，列为OTU。
    subset_size (int): 每个抽取的子集应包含的样本数量。
    num_subsets (int, optional): 要抽取的子集数量。默认为5。
    output_folder (str, optional): 用于存储生成的BIOM文件的文件夹名称。默认为 "biom_subsets"。
    random_seed (int, optional): 用于随机数生成的种子，以保证结果可复现。默认为42。

    返回:
    list: 一个包含所有成功生成的BIOM文件路径的列表。
    """

    # 0. 输入验证
    if not isinstance(df_abundance, pd.DataFrame):
        raise ValueError("输入 'df_abundance' 必须是一个 Pandas DataFrame。")
    if df_abundance.empty:
        raise ValueError("输入的DataFrame为空。")
    if not isinstance(subset_size, int) or subset_size <= 0:
        raise ValueError("'subset_size' 必须是一个正整数。")
    if not isinstance(num_subsets, int) or num_subsets <= 0:
        raise ValueError("'num_subsets' 必须是一个正整数。")
    # if df_abundance.shape[0] < subset_size * num_subsets:
    #     raise ValueError(
    #         f"DataFrame中的总样本数 ({df_abundance.shape[0]}) "
    #         f"不足以创建 {num_subsets} 个大小为 {subset_size} 的无重叠子集。"
    #         f"至少需要 {subset_size * num_subsets} 个样本。"
    #     )

    # 1. 预处理：填充NaN并设定随机种子
    print("步骤1: 预处理DataFrame...")
    df_filled = df_abundance.fillna(0)
    print(f" - NaN值已用0填充。")

    # 使用NumPy的随机状态以更好地控制种子，或者直接用np.random.seed()
    # np.random.seed(random_seed) # 全局设定
    # 或者创建一个RandomState实例
    rng = np.random.RandomState(random_seed)
    print(f" - 随机种子已设置为: {random_seed}")

    # 2. 获取并打乱样本ID（行索引）
    all_sample_ids = df_filled.index.tolist()
    # shuffled_sample_ids = np.random.permutation(all_sample_ids) # 如果使用全局np.random.seed
    shuffled_sample_ids = rng.permutation(all_sample_ids) # 使用rng实例的permutation

    # 3. 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)
    print(f"\n步骤2: BIOM文件将保存到文件夹: '{output_folder}'")

    generated_files = []

    # 4. 抽取子集并保存为BIOM文件
    print(f"\n步骤3: 开始抽取 {num_subsets} 个子集，每个子集包含 {subset_size} 个样本...")
    for i in range(num_subsets):
        start_index = i * subset_size
        end_index = (i + 1) * subset_size
        
        current_subset_sample_ids = shuffled_sample_ids[start_index:end_index]
        df_subset = df_filled.loc[current_subset_sample_ids]
        
        print(f"\n  正在处理子集 {i+1}/{num_subsets} (样本数: {len(df_subset)})...")

        # 将DataFrame转换为BIOM格式所需的输入
        # BIOM Table构造函数期望数据是 (observations x samples)
        # 我们的df_subset是 (samples x observations/OTUs)
        data_matrix = df_subset.values.T  # 转置
        observation_ids = df_subset.columns.tolist() # OTU IDs
        sample_ids_for_biom = df_subset.index.tolist() # 当前子集的Sample IDs

        # 创建BIOM Table对象
        try:
            table = biom.Table(
                data=data_matrix,
                observation_ids=observation_ids,
                sample_ids=sample_ids_for_biom
            )
            
            # 定义输出文件名
            biom_filename = os.path.join(output_folder, f"subset_{i+1}.biom")
            
            # 将BIOM Table对象写入HDF5文件
            # 'w'模式会覆盖已存在的文件
            with h5py.File(biom_filename, 'w') as hf:
                table.to_hdf5(hf, generated_by=f"create_biom_subsets_function_subset_{i+1}")
            
            print(f"    子集 {i+1} 已成功保存为BIOM文件: {biom_filename}")
            generated_files.append(biom_filename)

        except Exception as e:
            print(f"    错误：创建或保存子集 {i+1} 的BIOM文件时失败: {e}")
            print(f"      数据矩阵维度: {data_matrix.shape}")
            print(f"      观察ID数量: {len(observation_ids)}")
            print(f"      样本ID数量: {len(sample_ids_for_biom)}")


    print(f"\n处理完毕。总共生成了 {len(generated_files)} 个BIOM文件。")
    return generated_files


In [6]:
created_files = create_and_save_biom_subsets(df_abundance=df_table_all,subset_size=1000,num_subsets=5,output_folder='Data/data_subset/size_1000',random_seed=42)


步骤1: 预处理DataFrame...
 - NaN值已用0填充。
 - 随机种子已设置为: 42

步骤2: BIOM文件将保存到文件夹: 'data_subset/size_1000'

步骤3: 开始抽取 5 个子集，每个子集包含 1000 个样本...

  正在处理子集 1/5 (样本数: 1000)...
    子集 1 已成功保存为BIOM文件: data_subset/size_1000/subset_1.biom

  正在处理子集 2/5 (样本数: 1000)...
    子集 2 已成功保存为BIOM文件: data_subset/size_1000/subset_2.biom

  正在处理子集 3/5 (样本数: 1000)...
    子集 3 已成功保存为BIOM文件: data_subset/size_1000/subset_3.biom

  正在处理子集 4/5 (样本数: 1000)...
    子集 4 已成功保存为BIOM文件: data_subset/size_1000/subset_4.biom

  正在处理子集 5/5 (样本数: 1000)...
    子集 5 已成功保存为BIOM文件: data_subset/size_1000/subset_5.biom

处理完毕。总共生成了 5 个BIOM文件。


In [7]:
created_files = create_and_save_biom_subsets(df_abundance=df_table_all,subset_size=2000,num_subsets=5,output_folder='Data/data_subset/size_2000',random_seed=42)


步骤1: 预处理DataFrame...


 - NaN值已用0填充。
 - 随机种子已设置为: 42

步骤2: BIOM文件将保存到文件夹: 'data_subset/size_2000'

步骤3: 开始抽取 5 个子集，每个子集包含 2000 个样本...

  正在处理子集 1/5 (样本数: 2000)...
    子集 1 已成功保存为BIOM文件: data_subset/size_2000/subset_1.biom

  正在处理子集 2/5 (样本数: 2000)...
    子集 2 已成功保存为BIOM文件: data_subset/size_2000/subset_2.biom

  正在处理子集 3/5 (样本数: 2000)...
    子集 3 已成功保存为BIOM文件: data_subset/size_2000/subset_3.biom

  正在处理子集 4/5 (样本数: 2000)...
    子集 4 已成功保存为BIOM文件: data_subset/size_2000/subset_4.biom

  正在处理子集 5/5 (样本数: 2000)...
    子集 5 已成功保存为BIOM文件: data_subset/size_2000/subset_5.biom

处理完毕。总共生成了 5 个BIOM文件。


In [8]:
created_files = create_and_save_biom_subsets(df_abundance=df_table_all,subset_size=5000,num_subsets=5,output_folder='Data/data_subset/size_5000',random_seed=42)


步骤1: 预处理DataFrame...


 - NaN值已用0填充。
 - 随机种子已设置为: 42

步骤2: BIOM文件将保存到文件夹: 'data_subset/size_5000'

步骤3: 开始抽取 5 个子集，每个子集包含 5000 个样本...

  正在处理子集 1/5 (样本数: 5000)...
    子集 1 已成功保存为BIOM文件: data_subset/size_5000/subset_1.biom

  正在处理子集 2/5 (样本数: 5000)...
    子集 2 已成功保存为BIOM文件: data_subset/size_5000/subset_2.biom

  正在处理子集 3/5 (样本数: 5000)...
    子集 3 已成功保存为BIOM文件: data_subset/size_5000/subset_3.biom

  正在处理子集 4/5 (样本数: 5000)...
    子集 4 已成功保存为BIOM文件: data_subset/size_5000/subset_4.biom

  正在处理子集 5/5 (样本数: 5000)...
    子集 5 已成功保存为BIOM文件: data_subset/size_5000/subset_5.biom

处理完毕。总共生成了 5 个BIOM文件。


In [9]:
created_files = create_and_save_biom_subsets(df_abundance=df_table_all,subset_size=10000,num_subsets=5,output_folder='Data/data_subset/size_10000',random_seed=42)


步骤1: 预处理DataFrame...
 - NaN值已用0填充。
 - 随机种子已设置为: 42

步骤2: BIOM文件将保存到文件夹: 'data_subset/size_10000'

步骤3: 开始抽取 5 个子集，每个子集包含 10000 个样本...

  正在处理子集 1/5 (样本数: 10000)...
    子集 1 已成功保存为BIOM文件: data_subset/size_10000/subset_1.biom

  正在处理子集 2/5 (样本数: 10000)...
    子集 2 已成功保存为BIOM文件: data_subset/size_10000/subset_2.biom

  正在处理子集 3/5 (样本数: 10000)...
    子集 3 已成功保存为BIOM文件: data_subset/size_10000/subset_3.biom

  正在处理子集 4/5 (样本数: 10000)...
    子集 4 已成功保存为BIOM文件: data_subset/size_10000/subset_4.biom

  正在处理子集 5/5 (样本数: 10000)...
    子集 5 已成功保存为BIOM文件: data_subset/size_10000/subset_5.biom

处理完毕。总共生成了 5 个BIOM文件。


In [10]:
created_files = create_and_save_biom_subsets(df_abundance=df_table_all,subset_size=20000,num_subsets=5,output_folder='Data/data_subset/size_20000',random_seed=42)


步骤1: 预处理DataFrame...
 - NaN值已用0填充。
 - 随机种子已设置为: 42

步骤2: BIOM文件将保存到文件夹: 'data_subset/size_20000'

步骤3: 开始抽取 5 个子集，每个子集包含 20000 个样本...

  正在处理子集 1/5 (样本数: 20000)...
    子集 1 已成功保存为BIOM文件: data_subset/size_20000/subset_1.biom

  正在处理子集 2/5 (样本数: 20000)...
    子集 2 已成功保存为BIOM文件: data_subset/size_20000/subset_2.biom

  正在处理子集 3/5 (样本数: 20000)...
    子集 3 已成功保存为BIOM文件: data_subset/size_20000/subset_3.biom

  正在处理子集 4/5 (样本数: 20000)...
    子集 4 已成功保存为BIOM文件: data_subset/size_20000/subset_4.biom

  正在处理子集 5/5 (样本数: 20000)...
    子集 5 已成功保存为BIOM文件: data_subset/size_20000/subset_5.biom

处理完毕。总共生成了 5 个BIOM文件。


In [11]:
created_files = create_and_save_biom_subsets(df_abundance=df_table_all,subset_size=40000,num_subsets=5,output_folder='Data/data_subset/size_40000',random_seed=42)


步骤1: 预处理DataFrame...
 - NaN值已用0填充。
 - 随机种子已设置为: 42

步骤2: BIOM文件将保存到文件夹: 'data_subset/size_40000'

步骤3: 开始抽取 5 个子集，每个子集包含 40000 个样本...

  正在处理子集 1/5 (样本数: 40000)...
    子集 1 已成功保存为BIOM文件: data_subset/size_40000/subset_1.biom

  正在处理子集 2/5 (样本数: 40000)...
    子集 2 已成功保存为BIOM文件: data_subset/size_40000/subset_2.biom

  正在处理子集 3/5 (样本数: 40000)...
    子集 3 已成功保存为BIOM文件: data_subset/size_40000/subset_3.biom

  正在处理子集 4/5 (样本数: 40000)...
    子集 4 已成功保存为BIOM文件: data_subset/size_40000/subset_4.biom

  正在处理子集 5/5 (样本数: 40000)...
    子集 5 已成功保存为BIOM文件: data_subset/size_40000/subset_5.biom

处理完毕。总共生成了 5 个BIOM文件。


In [12]:
created_files = create_and_save_biom_subsets(df_abundance=df_table_all,subset_size=80000,num_subsets=5,output_folder='Data/data_subset/size_80000',random_seed=42)


步骤1: 预处理DataFrame...
 - NaN值已用0填充。
 - 随机种子已设置为: 42

步骤2: BIOM文件将保存到文件夹: 'data_subset/size_80000'

步骤3: 开始抽取 5 个子集，每个子集包含 80000 个样本...

  正在处理子集 1/5 (样本数: 80000)...
    子集 1 已成功保存为BIOM文件: data_subset/size_80000/subset_1.biom

  正在处理子集 2/5 (样本数: 80000)...
    子集 2 已成功保存为BIOM文件: data_subset/size_80000/subset_2.biom

  正在处理子集 3/5 (样本数: 80000)...
    子集 3 已成功保存为BIOM文件: data_subset/size_80000/subset_3.biom

  正在处理子集 4/5 (样本数: 80000)...
    子集 4 已成功保存为BIOM文件: data_subset/size_80000/subset_4.biom

  正在处理子集 5/5 (样本数: 80000)...
    子集 5 已成功保存为BIOM文件: data_subset/size_80000/subset_5.biom

处理完毕。总共生成了 5 个BIOM文件。


In [13]:
created_files = create_and_save_biom_subsets(df_abundance=df_table_all,subset_size=160000,num_subsets=5,output_folder='Data/data_subset/size_160000',random_seed=42)


步骤1: 预处理DataFrame...
 - NaN值已用0填充。
 - 随机种子已设置为: 42

步骤2: BIOM文件将保存到文件夹: 'data_subset/size_160000'

步骤3: 开始抽取 5 个子集，每个子集包含 160000 个样本...

  正在处理子集 1/5 (样本数: 160000)...
    子集 1 已成功保存为BIOM文件: data_subset/size_160000/subset_1.biom

  正在处理子集 2/5 (样本数: 160000)...
    子集 2 已成功保存为BIOM文件: data_subset/size_160000/subset_2.biom

  正在处理子集 3/5 (样本数: 160000)...
    子集 3 已成功保存为BIOM文件: data_subset/size_160000/subset_3.biom

  正在处理子集 4/5 (样本数: 160000)...
    子集 4 已成功保存为BIOM文件: data_subset/size_160000/subset_4.biom

  正在处理子集 5/5 (样本数: 160000)...
    子集 5 已成功保存为BIOM文件: data_subset/size_160000/subset_5.biom

处理完毕。总共生成了 5 个BIOM文件。


In [14]:
created_files = create_and_save_biom_subsets(df_abundance=df_table_all,subset_size=200000,num_subsets=5,output_folder='Data/data_subset/size_200000',random_seed=42)


步骤1: 预处理DataFrame...
 - NaN值已用0填充。
 - 随机种子已设置为: 42

步骤2: BIOM文件将保存到文件夹: 'data_subset/size_200000'

步骤3: 开始抽取 5 个子集，每个子集包含 200000 个样本...

  正在处理子集 1/5 (样本数: 200000)...
    子集 1 已成功保存为BIOM文件: data_subset/size_200000/subset_1.biom

  正在处理子集 2/5 (样本数: 200000)...
    子集 2 已成功保存为BIOM文件: data_subset/size_200000/subset_2.biom

  正在处理子集 3/5 (样本数: 200000)...
    子集 3 已成功保存为BIOM文件: data_subset/size_200000/subset_3.biom

  正在处理子集 4/5 (样本数: 200000)...
    子集 4 已成功保存为BIOM文件: data_subset/size_200000/subset_4.biom

  正在处理子集 5/5 (样本数: 199876)...
    子集 5 已成功保存为BIOM文件: data_subset/size_200000/subset_5.biom

处理完毕。总共生成了 5 个BIOM文件。
